# Training of the Hierarchical model for miniwob++

In [1]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install datasets transformers rouge-score nltk evaluate accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from datasets import load_dataset

# Load dataset regarding action-step sequences
raw_datasets_actions = load_dataset("LucasThil/miniwob_plusplus_hierarchical_training_actions")

# Load dataset regarding task of hierarchical planning
raw_datasets_planning = load_dataset("LucasThil/miniwob_plusplus_hierarchical_planning")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
raw_datasets_actions, raw_datasets_planning

(DatasetDict({
     train: Dataset({
         features: ['history_episodes', 'instruction', 'actions', 'refs', 'keydown_text', 'subtask_completion'],
         num_rows: 42097
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['hierarchical_plans', 'standolone_instruction'],
         num_rows: 10960
     })
 }))

In [6]:
# Add extra column regarding condensed target_labels
# We append together the (action, ref, text)
# Where action is either of type 'click' or 'type'
# ref is the targeted element number to perform the action on
# and type is the text content in case the action is 'type'
def condensed_targets(dataset, label):
  condensed_targets = []
  for row in dataset[label]:
    target_text = row['keydown_text']
    if target_text is None:
      target_text = ""
    target = '{' + row['actions'] + ', ' + str(row['refs']) + ', ' + target_text + ', ' + row['subtask_completion'] + '}'
    condensed_targets.append(target)
  return condensed_targets

raw_datasets_actions = raw_datasets_actions.map(lambda example: {k: v if v == v else "" for k, v in example.items()})
raw_datasets_actions['train'] = raw_datasets_actions['train'].add_column("targets", condensed_targets(raw_datasets_actions, 'train'))
raw_datasets_actions['train'] = raw_datasets_actions['train'].remove_columns(['actions', 'refs', 'keydown_text', 'instruction', 'subtask_completion'])
raw_datasets_actions

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 42097
    })
})

In [7]:
# Format the hierchical planning dataset, later we'll marge it in the action dataset
# First add the Command specifying that this row is about asking the Model to Devise a plan
raw_datasets_planning = raw_datasets_planning.map(lambda row: {'standolone_instruction': 'Devise a plan for the following instruction: ' + row['standolone_instruction']})  
raw_datasets_planning = raw_datasets_planning.map(lambda example: {k: v if v == v else "" for k, v in example.items()})

# Rename the columns
raw_datasets_planning = raw_datasets_planning.rename_column('standolone_instruction', 'history_episodes')
raw_datasets_planning = raw_datasets_planning.rename_column('hierarchical_plans', 'targets')
raw_datasets_planning

DatasetDict({
    train: Dataset({
        features: ['targets', 'history_episodes'],
        num_rows: 10960
    })
})

In [8]:
from datasets import Dataset
from datasets import concatenate_datasets, DatasetDict

# select how much of the datset we want to use
# shuffle the elements, and split for a test set
def reduce_split_dataset(raw_datasets, coef):
  raw_datasets['train'] = raw_datasets['train'].shuffle()
  raw_datasets['train'] = raw_datasets["train"].select(range(int(len(raw_datasets["train"])*coef)))
  
  raw_datasets = raw_datasets["train"].train_test_split(train_size=0.95, seed=20)
  val_dataset = raw_datasets.pop("test")
  raw_datasets = raw_datasets["train"].train_test_split(train_size=0.95, seed=20)
  raw_datasets.update({'validation': val_dataset})

  return raw_datasets
  

In [9]:
raw_datasets_actions = reduce_split_dataset(raw_datasets_actions, 0.05)
raw_datasets_planning = reduce_split_dataset(raw_datasets_planning, 0.05)
raw_datasets_actions, raw_datasets_planning

(DatasetDict({
     train: Dataset({
         features: ['history_episodes', 'targets'],
         num_rows: 1898
     })
     test: Dataset({
         features: ['history_episodes', 'targets'],
         num_rows: 100
     })
     validation: Dataset({
         features: ['history_episodes', 'targets'],
         num_rows: 106
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['targets', 'history_episodes'],
         num_rows: 494
     })
     test: Dataset({
         features: ['targets', 'history_episodes'],
         num_rows: 26
     })
     validation: Dataset({
         features: ['targets', 'history_episodes'],
         num_rows: 28
     })
 }))

In [10]:

# Now combine both datasets in one
train_dataset = concatenate_datasets([raw_datasets_actions['train'], raw_datasets_planning['train']])
test_dataset = concatenate_datasets([raw_datasets_actions['test'], raw_datasets_planning['test']])
validate_dataset = concatenate_datasets([raw_datasets_actions['validation'], raw_datasets_planning['validation']])
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validate_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 2392
    })
    test: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 126
    })
    validation: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 134
    })
})

In [11]:
# Explore the dataset to ensure there are no NaNs, or lengths too big

# Show NaNs
nan_examples = dataset['train'].filter(lambda example: any(val != val for val in example.values()))

# Print the NaN examples
for example in nan_examples:
    print(f"Example ID: {example['id']}")
    for key, value in example.items():
        if value != value:  # Check for NaN values
            print(f"{key}: {value}")
    print('\n')

short_examples = dataset['train'].filter(lambda example: any(len(str(val)) < 5 for val in example.values()))

# Print the short examples
for example in short_examples:
    print(f"Example ID: {example}")
    for key, value in example.items():
        if len(str(value)) < 5:  # Check for short string values
            print(f"{key}: {value}")
    print('\n')
    break

Filter:   0%|          | 0/2392 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2392 [00:00<?, ? examples/s]

In [12]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(dataset["train"])

# Prepare Model

In [14]:
from transformers import AutoTokenizer

checkpoint = "google/t5-v1_1-base"
#checkpoint = 'LucasThil/T5_base_hierarchy1'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [15]:
tokenizer

T5TokenizerFast(name_or_path='google/t5-v1_1-base', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>'

In [16]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["history_episodes"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    #with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["targets"], max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[13218, 344, 8, 3808, 7, 12, 253, 8, 1309, 96, 553, 155, 9, 15, 1280, 3, 2, 6965, 6273, 17592, 2266, 121, 3155, 2, 8481, 3, 23, 26, 17592, 210, 5846, 121, 6273, 17592, 2368, 121, 3155, 2, 8481, 3, 23, 26, 17592, 498, 121, 2287, 17592, 76, 23, 18, 10309, 7, 3, 76, 23, 18, 13165, 49, 18, 1748, 3, 76, 23, 18, 12018, 2782, 3, 76, 23, 18, 12018, 2782, 18, 14819, 121, 6273, 17592, 2596, 121, 3155, 2, 83, 2287, 17592, 76, 23, 18, 10309, 7, 18, 14128, 3, 76, 23, 18, 13165, 49, 18, 1748, 3, 76, 23, 18, 15061, 49, 18, 60, 2244, 3, 76, 23, 18, 15061, 49, 18, 2482, 291, 12304, 3, 76, 23, 18, 12018, 2782, 18, 3313, 49, 121, 6273, 17592, 17395, 3155, 2, 40, 23, 2287, 17592, 76, 23, 18, 10309, 7, 18, 10309, 3, 76, 23, 18, 13165, 49, 18, 2916, 3, 76, 23, 18, 5540, 18, 31026, 3, 76, 23, 18, 10309, 3, 76, 23, 18, 10309, 7, 18, 6645, 3, 76, 23, 18, 5540, 18, 6645, 121, 6273, 17592, 2555, 121, 3155, 2, 9, 3, 23, 26, 17592, 76, 23, 18, 23, 26, 9169, 121, 2287, 17592, 76, 23, 18, 10309, 7, 18

In [18]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2392 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

In [19]:
print(len(tokenized_datasets['train'][0]['input_ids']), len(tokenized_datasets['train'][1]['input_ids']))


1024 1024


In [20]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# new:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [21]:
import evaluate
# Use ROUGE as a metric
rouge = evaluate.load("rouge")

In [22]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [23]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.config.ctc_zero_infinity = True

In [24]:
trainer = ""
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

69

In [26]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-v1_1-base_hierarchy1",
    evaluation_strategy="epoch",
    learning_rate=2e-6,
    #per_device_train_batch_size=16,
    #per_device_eval_batch_size=16,
    auto_find_batch_size=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    #fp16_full_eval=True,
    push_to_hub=False,
    #logging_nan_inf_filter=True,
    dataloader_drop_last=True,
    #optim='adamw_torch'
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,nan,0.040500,0.017300,0.036000,0.036300,19.000000


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 30>:30                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:1662 in train                     │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/accelerate/utils/memory.py:124 in decorator               │
│                                                                                                  │
│   121 │   │   │   if batch_size == 0:                                                            │
│   122 │   │   │   │   raise RuntimeError("No executable batch size found, reached zero.")        │
│   123 │   │   │   try:                                                                           │
│ ❱ 124 │   │   │   │   return function(batch_size, *args, **kwargs)                               │
│   125 │   │   │   except Exception as e:                                                         │
│   126 │   │   │   │   if should_reduce_batch_size(e):                                            │
│   127 │   │   │   │   │   gc.collect()                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:1929 in _inner_training_loop      │
│                                                                                                  │
│   1926 │   │   │   │   │   with model.no_sync():                                                 │
│   1927 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1928 │   │   │   │   else:                                                                     │
│ ❱ 1929 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1930 │   │   │   │                                                                             │
│   1931 │   │   │   │   if (                                                                      │
│   1932 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:2709 in training_step             │
│                                                                                                  │
│   2706 │   │   │   loss = loss / self.args.gradient_accumulation_steps                           │
│   2707 │   │                                                                                     │
│   2708 │   │   if self.do_grad_scaling:                                                          │
│ ❱ 2709 │   │   │   self.scaler.scale(loss).backward()                                            │
│   2710 │   │   elif self.use_apex:                         

In [ ]:
trainer.push_to_hub()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:3639 in push_to_hub               │
│                                                                                                  │
│   3636 │   │   # If a user calls manually `push_to_hub` with `self.args.push_to_hub = False`, w  │
│   3637 │   │   # it might fail.                                                                  │
│   3638 │   │   if not hasattr(self, "repo"):                                                     │
│ ❱ 3639 │   │   │   self.init_git_repo()                                                          │
│   3640 │   │                                                                                     │
│   3641 │   │   model_name = kwargs.pop("model_name", None)                                       │
│   3642 │   │   if model_name is None and self.args.should_save:                                  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:3497 in init_git_repo             │
│                                                                                                  │
│   3494 │   │   # Make sure the repo exists.                                                      │
│   3495 │   │   create_repo(repo_name, token=self.args.hub_token, private=self.args.hub_private_  │
│   3496 │   │   try:                                                                              │
│ ❱ 3497 │   │   │   self.repo = Repository(self.args.output_dir, clone_from=repo_name, token=sel  │
│   3498 │   │   except EnvironmentError:                                                          │
│   3499 │   │   │   if self.args.overwrite_output_dir and at_init:                                │
│   3500 │   │   │   │   # Try again after wiping output_dir                                       │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn     │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                                                         │
│   121 │                                                                                          │
│   122 │   return _inner_fn  # type: ignore                                                       │
│   123                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/huggingface_hub/repository.py:516 in __init__             │
│                                                                                                  │
│    513 │   │   │   self.huggingface_token = HfFolder.get_token()                                 │
│    514 │   │                                                                                     │
│    515 │   │   if clone_from is not None:                                                        │
│ ❱  516 │   │   │   self.clone_from(repo_url=clone_from)                                          │
│    517 │   │   else:                                       

In [ ]:
trainer.save_model("LucasThil/T5_base_hierarchy3")
trainer.save_config("LucasThil/T5_base_hierarchy3")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Seq2SeqTrainer' object has no attribute 'save_config'

In [ ]:
!transformers-cli upload LucasThil/T5_base_hierarchy3

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_shell.py:98 in system                       │
│                                                                                                  │
│    95 │   if pip_warn:                                                                           │
│    96 │     kwargs.update({'also_return_output': True})                                          │
│    97 │                                                                                          │
│ ❱  98 │   output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=pr   │
│    99 │                                                                                          │
│   100 │   if pip_warn:                                                                           │
│   101 │     _pip.print_previous_import_warning(output)                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_system_commands.py:453 in _system_compat    │
│                                                                                                  │
│   450   # We set a higher depth than the IPython system command since a shell object             │
│   451   # is expected to call this function, thus adding one level of nesting to the             │
│   452   # stack.                                                                                 │
│ ❱ 453   result = _run_command(                                                                   │
│   454 │     shell.var_expand(cmd, depth=2), clear_streamed_output=False                          │
│   455   )                                                                                        │
│   456   shell.user_ns['_exit_code'] = result.returncode                                          │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_system_commands.py:167 in _run_command      │
│                                                                                                  │
│   164   """Calls the shell command, forwarding input received on the stdin_socket."""            │
│   165   locale_encoding = locale.getpreferredencoding()                                          │
│   166   if locale_encoding != _ENCODING:                                                         │
│ ❱ 167 │   raise NotImplementedError(                                                             │
│   168 │   │   'A UTF-8 locale is required. Got {}'.format(locale_encoding)                       │
│   169 │   )                                                                                      │
│   170                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
model = ""
trainer = ""

In [ ]:
tokenized_datasets['train'][0]['targets']

'{click, 14, , stop}'

In [ ]:
# Test compute metrics
pred = tokenized_datasets['train'][0]['labels']
reference = tokenized_datasets['train'][0]['targets']
pred_labels = [pred, pred]
print(compute_metrics(pred_labels))
decoded_pred = tokenizer.batch_decode(pred, skip_special_tokens=True)
print(decoded_pred)

result = rouge.compute(predictions=decoded_pred, references=decoded_pred, use_stemmer=True)
print(result)
prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for p in pred]
print(prediction_lens)
result["gen_len"] = np.mean(prediction_lens)
print(result)

{'rouge1': 0.0714, 'rouge2': 0.0, 'rougeL': 0.0714, 'rougeLsum': 0.0714, 'gen_len': 0.2381}
['', '', 'click', ',', '14,', '', ',', 'stop', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
{'rouge1': 0.07142857142857142, 'rouge2': 0.0, 'rougeL': 0.07142857142857142, 'rougeLsum': 0.07142857142857142}
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
{'rouge1': 0.07142857142857142, 'rouge2': 0.0, 'rougeL': 0.07142857142857142, 'rougeLsum': 0.07142857142857142, 'gen_len': 1.0}


# Test Tensorflow

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="tf_T5_hierarchical_1",
    tokenizer=tokenizer,
)

/content/tf_T5_hierarchical_1 is already a clone of https://huggingface.co/LucasThil/tf_T5_hierarchical_1. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=callbacks)

Epoch 1/3
2992/2992 [==============================] - ETA: 0s - loss: 0.2824

ValueError: ignored

In [ ]:
model = ""

In [ ]:
optimizer = ""

# Inference

In [ ]:
text = "Devise a plan for the following instruction: Select 12rh2X4, Hjfr, iMDDWUP, Jc0Wr4A, Rg and click Submit.	"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/T5_base_hierarchy4/checkpoint-10500")
inputs = tokenizer(text, return_tensors="pt").input_ids

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/content/T5_base_hierarchy4/checkpoint-10500")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

''